In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [62]:
train = pd.read_csv('https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Black_friday/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Black_friday/test.csv')

In [63]:
print("train :" ,train.shape)
print("test :" ,test.shape)

train : (550068, 12)
test : (233599, 11)


In [64]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [65]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [66]:
((train.isnull().sum())/train.shape[0])*100

User_ID                        0.000000
Product_ID                     0.000000
Gender                         0.000000
Age                            0.000000
Occupation                     0.000000
City_Category                  0.000000
Stay_In_Current_City_Years     0.000000
Marital_Status                 0.000000
Product_Category_1             0.000000
Product_Category_2            31.566643
Product_Category_3            69.672659
Purchase                       0.000000
dtype: float64

In [67]:
((test.isnull().sum())/test.shape[0])*100

User_ID                        0.000000
Product_ID                     0.000000
Gender                         0.000000
Age                            0.000000
Occupation                     0.000000
City_Category                  0.000000
Stay_In_Current_City_Years     0.000000
Marital_Status                 0.000000
Product_Category_1             0.000000
Product_Category_2            30.969311
Product_Category_3            69.590195
dtype: float64

In [68]:
test.fillna(0,inplace=True)
train.fillna(0,inplace=True)

In [69]:
train.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.500680e+05,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000,550068.000000
mean,1.003029e+06,8.076707,0.409653,5.404270,6.735436,3.841941,9263.968713
std,1.727592e+03,6.522660,0.491770,3.936211,6.215492,6.250712,5023.065394
min,1.000001e+06,0.000000,0.000000,1.000000,0.000000,0.000000,12.000000
25%,1.001516e+06,2.000000,0.000000,1.000000,0.000000,0.000000,5823.000000
50%,1.003077e+06,7.000000,0.000000,5.000000,5.000000,0.000000,8047.000000
75%,1.004478e+06,14.000000,1.000000,8.000000,14.000000,8.000000,12054.000000
max,1.006040e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23961.000000


In [70]:
col_name = train.columns
col_name

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [71]:
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [72]:
for features in col_name:
    print(features,'-->',len(train[features].unique()),':', train[features].unique())

User_ID --> 5891 : [1000001 1000002 1000003 ... 1004113 1005391 1001529]
Product_ID --> 3631 : ['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853']
Gender --> 2 : ['F' 'M']
Age --> 7 : ['0-17' '55+' '26-35' '46-50' '51-55' '36-45' '18-25']
Occupation --> 21 : [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6]
City_Category --> 3 : ['A' 'C' 'B']
Stay_In_Current_City_Years --> 5 : ['2' '4+' '3' '1' '0']
Marital_Status --> 2 : [0 1]
Product_Category_1 --> 20 : [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19]
Product_Category_2 --> 18 : [ 0.  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.]
Product_Category_3 --> 16 : [ 0. 14. 17.  5.  4. 16. 15.  8.  9. 13.  6. 12.  3. 18. 11. 10.]
Purchase --> 18105 : [ 8370 15200  1422 ...   135   123   613]


In [73]:
gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':17, '18-25':25, '26-35':35, '36-45':45, '46-50':50, '51-55':55, '55+':60}
city_dict = {'A':0, 'B':1, 'C':2}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}

train['Gender'] = train['Gender'].map(gender_dict)
test['Gender'] = test['Gender'].map(gender_dict)

train['Age'] = train['Age'].map(age_dict)
test['Age'] = test['Age'].map(age_dict)

train['City_Category'] = train['City_Category'].map(city_dict)
test['City_Category'] = test['City_Category'].map(city_dict)

train['Stay_In_Current_City_Years'] = train['Stay_In_Current_City_Years'].map(stay_dict)
test['Stay_In_Current_City_Years'] = test['Stay_In_Current_City_Years'].map(stay_dict)


In [74]:
testRes = test[['User_ID']]
testRes['Product_ID'] = test['Product_ID']
testRes

C:\Users\Nik\AppData\Local\Temp\ipykernel_12956\4082465640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testRes['Product_ID'] = test['Product_ID']


,User_ID,Product_ID
0,1000004,P00128942
1,1000009,P00113442
2,1000010,P00288442
3,1000010,P00145342
4,1000011,P00053842
...,...,...
233594,1006036,P00118942
233595,1006036,P00254642
233596,1006036,P00031842
233597,1006037,P00124742


In [75]:
for features in col_name:
    print(features,'-->',len(train[features].unique()),':', train[features].unique())

User_ID --> 5891 : [1000001 1000002 1000003 ... 1004113 1005391 1001529]
Product_ID --> 3631 : ['P00069042' 'P00248942' 'P00087842' ... 'P00370293' 'P00371644'
 'P00370853']
Gender --> 2 : [0 1]
Age --> 7 : [17 60 35 50 55 45 25]
Occupation --> 21 : [10 16 15  7 20  9  1 12 17  0  3  4 11  8 19  2 18  5 14 13  6]
City_Category --> 3 : [0 2 1]
Stay_In_Current_City_Years --> 5 : [2 4 3 1 0]
Marital_Status --> 2 : [0 1]
Product_Category_1 --> 20 : [ 3  1 12  8  5  4  2  6 14 11 13 15  7 16 18 10 17  9 20 19]
Product_Category_2 --> 18 : [ 0.  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.]
Product_Category_3 --> 16 : [ 0. 14. 17.  5.  4. 16. 15.  8.  9. 13.  6. 12.  3. 18. 11. 10.]
Purchase --> 18105 : [ 8370 15200  1422 ...   135   123   613]


In [76]:
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                          int64
Age                             int64
Occupation                      int64
City_Category                   int64
Stay_In_Current_City_Years      int64
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

plt.figure(figsize = (10, 10))
heatmap = sns.heatmap(train.corr(),annot = True)
heatmap.set_title('Correlation Heatmap', fontdict = {'fontsize' : 12}, pad = 12)

In [77]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
scaler = StandardScaler()

train['User_ID']=enc.fit_transform(train['User_ID'],)
test['User_ID']=enc.transform(test['User_ID'])

In [78]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,P00069042,0,17,10,0,2,0,3,0.0,0.0,8370
1,0,P00248942,0,17,10,0,2,0,1,6.0,14.0,15200
2,0,P00087842,0,17,10,0,2,0,12,0.0,0.0,1422
3,0,P00085442,0,17,10,0,2,0,12,14.0,0.0,1057
4,1,P00285442,1,60,16,2,4,0,8,0.0,0.0,7969


In [79]:
# Standard scalar
train['Product_ID'] = train['Product_ID'].str.replace('P00', '')
test['Product_ID'] = test['Product_ID'].str.replace('P00', '')

train['Product_ID'] = scaler.fit_transform(train['Product_ID'].values.reshape(-1, 1))
test['Product_ID'] = scaler.transform(test['Product_ID'].values.reshape(-1, 1))

# Label Encoding Product_IDs
new_product_ids = list(set(pd.unique(test_data['product_id'])) - set(pd.unique(train_data['product_id'])))

le = LabelEncoder()
train_data['product_id'] = le.fit_transform(train_data['product_id'])
test_data.loc[test_data['product_id'].isin(new_product_ids), 'product_id'] = -1
new_product_ids.append(-1)

test_data.loc[~test_data['product_id'].isin(new_product_ids), 'product_id'] = le.transform(test_data.loc[~test_data['product_id'].isin(new_product_ids), 'product_id'])

# Drop User ID and Product ID from test and train


In [80]:
for features in col_name:
    if len(train[features].unique())<4:
        print(features,'-->',len(train[features].unique()),':', train[features].unique())

Gender --> 2 : [0 1]
City_Category --> 3 : [0 2 1]
Marital_Status --> 2 : [0 1]


In [81]:
categorical_col = ['Gender', 'City_Category','Marital_Status']
numerical_col = ['Age', 'Occupation', 'Stay_In_Current_City_Years', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']

In [82]:
for col in categorical_col:
    train[col] = enc.fit_transform(train[col])
    test[col] = enc.transform(test[col])

In [83]:
for col in numerical_col:
    train[col] = scaler.fit_transform(train[col].values.reshape(-1, 1))
    test[col] = scaler.transform(test[col].values.reshape(-1, 1))

In [84]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,-1.028774,0,-2.059570,0.294864,0,0.109801,0,-0.610809,-1.083654,-0.614641,8370
1,0,0.722139,0,-2.059570,0.294864,0,0.109801,0,-1.118912,-0.118323,1.625106,15200
2,0,-0.845799,0,-2.059570,0.294864,0,0.109801,0,1.675656,-1.083654,-0.614641,1422
3,0,-0.869157,0,-2.059570,0.294864,0,0.109801,0,1.675656,1.168785,-0.614641,1057
4,1,1.077382,1,2.094842,1.214734,2,1.660861,0,0.659449,-1.083654,-0.614641,7969


In [85]:
train.dtypes

User_ID                         int64
Product_ID                    float64
Gender                          int64
Age                           float64
Occupation                    float64
City_Category                   int64
Stay_In_Current_City_Years    float64
Marital_Status                  int64
Product_Category_1            float64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [86]:
from sklearn.model_selection import train_test_split
X = train.drop(['Purchase'], axis=1)
y = train[['Purchase']]
X_test = test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

In [87]:
from collections import OrderedDict
model_performance = OrderedDict()

In [88]:
lr=linear_model.LinearRegression()
lm_model=lr.fit(X_train,y_train)

y_pred=lm_model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val,y_pred))

model_performance['Multi Linear Regression'] = rmse
print(f'Root Mean Squared Error of the model is : {rmse}')

Root Mean Squared Error of the model is : 4609.769799176727


In [89]:
xgb_reg = XGBRegressor(learning_rate=1.0, max_depth=6, min_child_weight=40, seed=0)

xgb_reg.fit(X_train, y_train)

y_pred = xgb_reg.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_pred, y_val))

print (xgb_reg)

model_performance['XgbRegression_6'] = rmse
print(f'Root Mean Squared Error of the model is : {rmse}')


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=1.0, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=40,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)
Root Mean Squared Error of the model is : 2591.851957429727


In [91]:
xgb_reg_1 = XGBRegressor(n_estimators=400, max_depth = 20, learning_rate = 0.05,  min_child_weight  = 40)

xgb_reg_1.fit(X_train, y_train)

y_pred = xgb_reg_1.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_pred, y_val))

print (xgb_reg_1)

model_performance['XgbRegression_10'] = rmse
print(f'Root Mean Squared Error of the model is : {rmse}')

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=20, max_leaves=0, min_child_weight=40,
             missing=nan, monotone_constraints='()', n_estimators=400, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)
Root Mean Squared Error of the model is : 2509.3071749707287


In [92]:
random_forest_regressor = RandomForestRegressor(n_estimators = 10, random_state = 27)
random_forest_regressor.fit(X_train, y_train)

Y_pred = random_forest_regressor.predict(X_val)
mse = mean_squared_error(y_val, Y_pred)
rmse = np.sqrt(mse)
model_performance['Random Forest_10'] = rmse
print(f'Root Mean Squared Error of the model is : {rmse}')

C:\Users\Nik\AppData\Local\Temp\ipykernel_12956\2842004990.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_regressor.fit(X_train, y_train)


Root Mean Squared Error of the model is : 2863.8589744718747


In [93]:
random_forest_regressor = RandomForestRegressor(n_estimators = 50, random_state = 27)
random_forest_regressor.fit(X_train, y_train)

Y_pred = random_forest_regressor.predict(X_val)
mse = mean_squared_error(y_val, Y_pred)
rmse = np.sqrt(mse)
model_performance['Random Forest_50'] = rmse
print(f'Root Mean Squared Error of the model is : {rmse}')

C:\Users\Nik\AppData\Local\Temp\ipykernel_12956\194075752.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest_regressor.fit(X_train, y_train)


Root Mean Squared Error of the model is : 2763.9505481180163


In [94]:
model_performance

OrderedDict([('Multi Linear Regression', 4609.769799176727),
             ('XgbRegression_6', 2591.851957429727),
             ('XgbRegression_10', 2509.3071749707287),
             ('Random Forest_10', 2863.8589744718747),
             ('Random Forest_50', 2763.9505481180163)])

In [ ]:
yPreds = xgb_reg_1.predict(X_test)
testRes['Purchase'] = yPreds
submission = testRes[['user_id', 'Product_ID', 'Purchase']]

submission.columns = ['User_ID', 'Product_ID', 'Purchase']
submission.to_csv('submission.csv', index = False)
submission.head()